In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


==============================================================

W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))

L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')

L1 = tf.nn.relu(L1)

L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

L1 = tf.nn.dropout(L1, keep_prob=0.7)


=============================================================

Layers를 사용

training = tf.placeholder(tf.bool)

conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)

pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], padding='SAME', strides=2)

dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7, training=training)

==============================================================

In [19]:
training_epochs = 15
batch_size = 100

In [36]:
tf.reset_default_graph() 

Y = tf.placeholder(tf.float32, [None,10])
X = tf.placeholder(tf.float32, [None,784])

training = tf.placeholder(tf.bool)

keep_prob = tf.Variable(0.7)

X_img = tf.reshape(X, [-1, 28, 28, 1])

# L1 Img In Shape = (?,28,28,1)
conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2, padding='SAME')
dropout1 = tf.layers.dropout(inputs=pool1, rate=keep_prob, training=training)

# L2 Img In Shape = (?,14,14,1)
conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2, padding='SAME')
dropout2 = tf.layers.dropout(inputs=pool2, rate=keep_prob, training=training)

# L3 Img In Shape = (?,7,7,1)
conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2], strides=2,padding='SAME')
dropout3 = tf.layers.dropout(inputs=pool3, rate=keep_prob, training=training)

layer = tf.reshape(dropout3, [-1,128*4*4])

# 
W4 = tf.get_variable("W4",shape=[128*4*4,625],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(layer,W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5",shape=[625,10],initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4,W5)+b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [37]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, training:True}
        c, _, = sess.run([cost,optimizer], feed_dict=feed_dict)
        avg_cost += c/total_batch
    print('Epoch : ', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch :  0001 cost = 0.724060966
Epoch :  0002 cost = 0.301145520
Epoch :  0003 cost = 0.229106103
Epoch :  0004 cost = 0.189259193
Epoch :  0005 cost = 0.162216295
Epoch :  0006 cost = 0.139651759
Epoch :  0007 cost = 0.124305094
Epoch :  0008 cost = 0.110518331
Epoch :  0009 cost = 0.099736418
Epoch :  0010 cost = 0.091047064
Epoch :  0011 cost = 0.082342297
Epoch :  0012 cost = 0.073570947
Epoch :  0013 cost = 0.070123282
Epoch :  0014 cost = 0.065811169
Epoch :  0015 cost = 0.060492892


In [38]:
prediction = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
print(sess.run(accuracy,feed_dict={X:mnist.test.images,Y:mnist.test.labels, training:False}))

0.9912
